In [2]:
import math
#import pint
#units = pint.UnitRegistry()

#units.define('dBw = 10 * math.log10(units.watt)')

#transmitter_power = 25 * units.watt
#transmitter_gain = transmitter_power.to(units.dBw)
#antenna_gain = 6 * units.dBw

# Link Budget

Mach 30 is designing a radio to receive transmissions from satellites (a ground station), named Ground Sphere. One of the most crucial calculations for determining whether the ground station's design will be able to receive the signals transmitted by a given satellite is the link budget. Simply put, a link budget factors in all gains and losses in the system composed of the transmitter (satellite) and receiver (ground station), and everything in between. 

The calculations themsleves are implemented by a python class named LinkBudget. The class models what are generally the most significant factors in calculating a link budget. Because there are many factors that are difficult or impossible to accurately factor in, such as the amount a given storm may introduce noise into the system, several aspects of the budget are averages or otherwise estimated values.

The overall equation for the Link Budget is Transmitted Power(dB) + Gains(dB) - Losses(dB). This Link Budget equation will further be broken down 

## Overview
 
For each test case presented, the LinkBudget class uses the broadcast frequency of the satellite radio system, as well as its total estimated power (gain) as a basis of calculation (more information on this will be provided later). Next, some trigonometry is performed to determine the distance the satellite might be from the ground station in an observation window to factor in propagation loss due to the signal spreading as it travels from the satellite to the ground staiton. This distance and the frequency are also used to determine the loss in signal strength due to it having to travel through air, otherwise known as the free space propagation loss. Possible noise generated by weather and other background sources such as microwave ovens are also factored in when applicable. Lastly, the capabilities of the ground station to amplify the signal are also taken into account. 


## Definitions

**gain** / **loss** - Gain refers to how well a component converts its input power to its output. For a radio, this describes its ability to convert its input power to output. For transmitting, its output is radio waves, and for receiving, this is the power of the signal as it reaches the radio. For a component of the system, this refers to how much it similarly increases (gain) or decreases (loss) the power from the radio to antenna, or the opposite.

**dB** - decibels. This is a unit to describe the power of a signal in reference to another value, such as Watts. It is tipically represented in reference to this other value (for example, dBw is decibels in reference ot Watts), often abbreviated to dB. The units are in a logarithmic scale, meaning 10dB is 10 times as powerful as 0, 20dB is 100 times as powerful,  etc.

## Broadcast Power

Whenever information is sent from one place to another, such as when a satellite transmits an image down to Earth to be received by a radio on the ground (ground station), the power (gain) of the signal being transmitted from the satellite is a critical factor in how well it will be able to be received by the ground station. This section of the document serves as a guide to determining what the gain of the source of the transmission may be. In some cases, the gain may be known, but in other cases it must be estimated based on knowledge of the components of the radio system used to transmit.

In the event that you do not know what the gain of the transmitter is, there are a few methods one might use to determine it. Some satellites have their broadcast frequency publicy available, such as with the NOAA satellites. This may be on the organizations website, but a great source is the Earth Observation Portal's [Satellite Mission Database](https://directory.eoportal.org/web/eoportal/satellite-missions), which contains detailed information about a wide number of satellites, all in one place. 

If you cannot access this information, you may have to make some estimates based upon known equipment of the transmitter, or simply make a guess for the purposes of calculating your link budget. If your attempting to calculate an estimate, knowing what the gain of the antenna, radio, and amplifier are is helpful, where you may then  be able to use a conservative estimate for the loss of other components in the system. This estimate may be 1.0db to 1.5db of loss in the 100MHz range (or higher for longer lengths of transmission line). This will decrease in higher frequencies due to the need for components of the transmission system such as filters or wave guides to be tightly matched to the transmission frequency.

### Example Gain Estimation ###

In [3]:
transmitter_power = 25.0
transmitter_gain = 10 * math.log10(transmitter_power)
antenna_gain = 6

For an example of this estimate, we will use a {{transmitter_power}}W transmitter, and an antenna with a wide signal dispersion and thus a low gain of {{antenna_gain}}dB gain. 

First, we must convert the transmitter's Watts to dBw so that we can calculate the overall gain of the transmission using the same units. Since dBw are a logarithmic scaled unit in reference to Watts, the conversion is simply 10 * log10(W), or 10 * log10({{transmitter_power}}), which is equal to {{round(transmitter_gain, 4)}} (rounded).

In [7]:
est_wire_gain = -0.05
est_connector_gain = -0.5
est_pointing_loss = -0.5

The second step is listing all of the other components involved in the transmission process. For this example, we will assume there is a transmission wire, a frequency filter (used to ensure that only a range or band of frequencies are transmitted), and associated connectors. As an estimate for a 10 meter (33 feet) transmission wire, it has a loss of {{abs(est_wire_gain)}}dBw. Note that this increases exponentially with the length of the wire. Our estimate for this example of the loss from the filter and connectors is an additional {{abs(est_connector_gain)}}dBw. Finally, we need to account for signal loss due to the antenna not being pointed directly at the receiver. Given the wide dispersion area of the signal from our antenna, we're estimating this at {{abs(est_pointing_loss)}}dBw. With values for the gain and loss of each of the components, we can now simply add them together to determine the transmission gain.

In [5]:
total_gain = transmitter_gain + est_connector_gain + est_wire_gain + antenna_gain + est_pointing_loss
total_gain_watts = pow(10, total_gain/10)

| Component | Gain (dBw) |
|---|--:|
| Transmitter | {{round(transmitter_gain, 4)}} |
| Connectors and Filter | {{est_connector_gain}} |
| Transmission Wire | {{est_wire_gain}} |
| Antenna | {{antenna_gain}} |
| Pointing Loss | {{est_pointing_loss}} |
|   | = |
| Total | {{round(total_gain, 4)}} |
The total gain of our example system of {{round(total_gain, 4)}}dBw, or {{round(total_gain_watts, 4)}}w, should be plenty for us to receive the signal it transmits in orbit on the ground. This was only an example though. Unless you know some details about the system being used to transmit, it may not be possible to do an estimation like this. Assuming you're doing an estimation like this for something that already exists, you will instead need to use some existing equipment to receive its signal and work backwards to calculate the gain.

## Calculating Distance from Transmitter to Receiver

Now that the transmitter is sending data, it has to travel to the receiver. As light travels from the transmitter to the receiver, it disperses, spreading out into a larger area than that of the transmitter. As a result, the distance the transmission travels incurs loss in the signal. There is also loss accumulated from traveling through a given medium (air for example), which we will account for in the next section. Calculating the distance from the transmitter to receiver so that we can determine the resulting loss involves a little bit of basic trigonometry.

### Known Variables ###
In the case of a satellite, the main piece of information you need to know is the orbital altitude of the satellite. Additionally, the altitude the receiver will be used at is important, but if unknown, the lowest altitude the receiver may be used at should be used (for example, sea level, or 0 meters). The last piece of the puzzle is the angle relative to the receiver the satellite will be as it rises above the horizon and the receiver starts to receive, as this is the angle at which the most signal loss will occur. 25° is a good value to use here, accounting for trees and other ground based obstructions. If you know the receiver is going to be used from some place without many ground based obstructions, such as a mountaintop, you might use a larger angle.

With that angle and the altitude of the satellite and receiver, we can calculate the slant range, or the distance between the transmitter to receiver, using basic trigonometry.  As the satellite travels over head, it is orbiting the Earth, and the receiver is on the ground. Both of these share the center of the Earth in common, acting as the third point of the triangle. Examining Diagram 1 below, you can see a visual representation of this, along with number of variables displayed. 

![alt text](diagram1.png "Diagram 1")
*<p style="text-align: center;">Diagram 1</p>*

The location of the transmitter (***A***), receiver (***B***), and center of the Earth (***C***) are shown, as well as the angle above the horizon the satellite will first be received from (***ϕ***). What needs to be solved for is the slant range, represented as ***c***. Some of these variables we have not yet assigned a value. The radius of the Earth (***r***) is known. Because the receiver is at sea level, this is also the distance from ***B*** to ***C***, represented in the diagram as ***a***. The satellite orbits Earth at a known altitude, and that plus ***r*** is equal to ***b***, or the distance from ***A*** to ***C***. If the receiver is not at sea level, ***a*** is similarly simply the altitude above sea level of the receiver plus ***r***

There are still some variables in the diagram that are unknown. These are ***β***, the angle at ***B*** between ***A*** and ***C***, ***θ***, the angle at ***C*** between ***A*** and ***B***, and similarly ***α***, the angle at ***A*** between ***B*** and ***C***. Lucky for us, just like ***b***, we have the information to calculate ***β***, with which we can calculate the other angles. Since ***A*** is at a known angle above the horizon, and the horizon is perpendicular to ***a***, ***β*** is the angle above the horizon + 90°.

### Calculating Slant Range ###
To solve for the remaining values, including the slant range, we need only two equations. The first is the law of sines, which states that for a triangle, the sine of an angle divided by the length of the opposite side of the triangle is equal to the same for each other angle. Below is a representation of this equation.

$$\frac{sin(α)}{a} = \frac{sin(β)}{b} = \frac{sin(θ)}{c}$$

With this equation, we can determine the value for ***α*** given the information we already know. 

$$\frac{sin(α)}{a} = \frac{sin(β)}{b}$$
$$sin(α) = a * \frac{sin(β)}{b}$$
$$sin(α) = \frac{a}{b} * sin(β)$$
$$α = sin^{-1}\bigg(\frac{a}{b} * sin(β)\bigg)$$

The other equation is the fact that the sum of the inner angles of a triangle are equal to 180. Below is a representation of this equation. 

$$180 = α + β + θ$$

With this equation, when two of the angles are known, the third can be calculated using subtraction. This equation is subsequently used to calculate ***θ***. With ***θ*** known, law of sines again can be used to calculate ***c***.

We can now represent all of the information we have, as well as the equations to solve for the as yet unknowns. Table 1 below lists all this information.

*<p style="text-align: center;">Table 1</p>*

| Variable | Description | Equation to Solve for Variable |
|---|---|---|---|
| ***A*** | Transmitter / Satellite |   |
| ***B*** | Receiver |   |
| ***C*** | Center of Earth |   |
| ***r*** | Earth Radius |   |
| ***a*** | Distance from ***C*** to *** B *** | $$r + B_{altitude}$$ |
| ***b*** | Distance from ***C*** to *** A *** | $$r + A_{altitude}$$ |
| ***c*** | Slant Range | $$sin(θ) * \frac{b}{sin(β)}$$ |
| ***ϕ*** | Angle of Transmitter above Horizon |   |
| ***α*** | Angle at ***A*** between ***B*** and ***C*** | $$sin^{-1}\bigg(\frac{a}{b} * sin(β)\bigg)$$ |
| ***β*** | Angle at ***B*** between ***A*** and ***C*** | $$θ + 90°$$ |
| ***θ*** | Angle at ***C*** between ***A*** and ***B*** | $$180 - α - β$$ |

## Calculating the Necessary Gain of the Reciever